In [1]:
import requests
import pandas as pd
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [2]:
stats_page = requests.get("http://champion.gg/statistics")
if stats_page.status_code ==200:
    stats_page = stats_page.text
else:
    raise ValueError("Error fetching stats: "+str(stats_page.status_code))

In [3]:
match_str = 'matchupData.stats = '
stat_data_start = stats_page[stats_page.find(match_str)+len(match_str):]
stat_data = stat_data_start[:stat_data_start.find(';')]

In [4]:
df = pd.read_json(StringIO(stat_data), orient='record')
df = df.set_index(['key'])
df = df.sort_index()
df = df.drop('title',axis =1)
df = df.drop('general',axis =1)
df.head()

,role
key,
Aatrox,Top
Ahri,Middle
Akali,Top
Akali,Middle
Alistar,Support


In [5]:
base_url = "http://champion.gg/champion/"
urls = [base_url+champ+"/"+role['role'] for champ, role in df.iterrows()]
exp_dict = {}
champ_list = [champ for champ,_ in df.iterrows()]
champ_1 = []
champ_2 = []
champ_3 = []
champ_4 = []
champ_5 = []

for url in urls:
    champ_page = requests.get(url)
    if champ_page.status_code == 200:
        champ_page = champ_page.text
    else:
        ValueError('Error requesting '+url+' status:'+champ_page.status_code)
    match_str = 'matchupData.championData = '
    champ_stats_start = champ_page[champ_page.find(match_str)+len(match_str):]
    match_str = '\"experienceRate\":'
    champ_stats_start = champ_stats_start[champ_stats_start.find(match_str)+len(match_str):]
    champ_stats = champ_stats_start[1:champ_stats_start.find(']')]
    champ_stats = [stat[1:-1] for stat in champ_stats.split(',')]
    champ_1 += [champ_stats[0]]
    champ_2 += [champ_stats[1]]
    champ_3 += [champ_stats[2]]
    champ_4 += [champ_stats[3]]
    champ_5 += [champ_stats[4]]
    pass

In [6]:
champ_df = pd.DataFrame(data={
    'Role': [role['role'] for _,role in df.iterrows()],
    '1-50': champ_1,
    '51-100': champ_2,
    '101-150': champ_3,
    '151-200': champ_4,
    '201+': champ_5
},index=champ_list)

In [7]:
champ_df.head()

,1-50,101-150,151-200,201+,51-100,Role
Aatrox,49.51,55.05,55.42,53.95,55.07,Top
Ahri,53.93,56.11,55.80,55.46,56.46,Middle
Akali,50.89,55.63,55.44,54.70,56.36,Top
Akali,50.40,55.53,55.76,54.41,56.36,Middle
Alistar,50.23,54.59,55.02,58.08,54.22,Support


In [9]:
champ_df.to_csv('data.csv',';')